# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.horizons import (
    horizons_bulk_vector_query,
    horizons_bulk_astrometry_query,
)

t0 = Time("2024-12-24T00:00:00", scale="utc")

In [2]:
horizons_obj = Horizons(id="274301", location="@0", epochs=[t0.jd])
horizons_table = horizons_obj.ephemerides(quantities="1,36,37", extra_precision=True)
horizons_coord = SkyCoord(
    ra=horizons_table["RA"][0], dec=horizons_table["DEC"][0], unit=(u.deg, u.deg)
)

horizons_table

targetname,datetime_str,datetime_jd,H,G,solar_presence,interfering_body,RA,DEC,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma
---,---,d,mag,---,---,---,deg,deg,arcsec,arcsec,arcsec,arcsec,deg,arcsec2
str28,str24,float64,float64,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64
274301 Wikipedia (2008 QH24),2024-Dec-24 00:00:00.000,2460668.5,16.98,0.15,,,136.766360849,11.336294782,0.066,0.049,0.066,0.049,-11.904,0.0101078


In [3]:
jorb_table = horizons_bulk_astrometry_query("274301", "500@0", t0)
jorb_coord = SkyCoord(ra=jorb_table["RA"], dec=jorb_table["DEC"], unit=(u.deg, u.deg))

jorb_table

,JD_UTC,RA,DEC,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma
0,2460668.5,136.766361,11.336295,0.066,0.049,0.066,0.049,-11.904,0.010108


In [4]:
jorb_coord.separation(horizons_coord).to(u.arcsec)

<Angle [0.] arcsec>

In [5]:
large_times = Time(t0.jd + jnp.linspace(0, 365, 1000), format="jd", scale="utc")
jorb_table = horizons_bulk_astrometry_query("274301", "500@0", large_times)
jorb_coord = SkyCoord(ra=jorb_table["RA"], dec=jorb_table["DEC"], unit=(u.deg, u.deg))[
    -10:
]
jorb_coord

<SkyCoord (ICRS): (ra, dec) in deg
    [(212.37610087, -9.48236651), (212.46622544, -9.50427124),
     (212.55640377, -9.52616316), (212.64663597, -9.54804218),
     (212.73692211, -9.56990825), (212.82726228, -9.59176127),
     (212.91765657, -9.61360117), (213.00810506, -9.63542787),
     (213.09860784, -9.6572413 ), (213.18916499, -9.67904138)]>

In [6]:
horizons_obj = Horizons(id="274301", location="@0", epochs=large_times.utc.jd[-10:])
horizons_table = horizons_obj.ephemerides(quantities="1,36,37", extra_precision=True)
horizons_coord = SkyCoord(
    ra=horizons_table["RA"], dec=horizons_table["DEC"], unit=(u.deg, u.deg)
)
horizons_coord

<SkyCoord (ICRS): (ra, dec) in deg
    [(212.37610087, -9.48236651), (212.46622544, -9.50427124),
     (212.55640377, -9.52616316), (212.64663597, -9.54804218),
     (212.73692211, -9.56990825), (212.82726228, -9.59176127),
     (212.91765657, -9.61360117), (213.00810506, -9.63542787),
     (213.09860784, -9.6572413 ), (213.18916499, -9.67904138)]>

In [7]:
horizons_coord.separation(jorb_coord).to(u.arcsec)

<Angle [3.55087423e-06, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00] arcsec>